In [1]:
path="/home/affine/vishnu/HOTEL_CLICK_BOOK_AGG_20150624.csv"

#creating a dataframe by loading the path of inputfile by using databricks library for csv files
pathdf = sqlCtx.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',delimiter=',').load(path)

In [2]:
#registering dataframe as temp tabel
pathdf.registerTempTable("new_ytd_feb14to23jun15")

In [3]:
#filter the outliers and selecting the required data
pathdf = pathdf.filter(pathdf['partner_pos'] == 'CA')
pathdf = pathdf.filter(pathdf['clicked_hotel_id'] <> 0)
pathdf = pathdf.filter(pathdf['xclick'] <> 0)


#selecting the required coloumns
tempdf = pathdf.select('local_date','partner_org','partner_pos','device_type','clicked_hotel_id','srch_window', \
            'srch_los','xclick','xcost','booked_flag','booked_hotel_id','booked_bkg_window', \
              'booked_los','lt_true_up_gp_usd','lt_true_up_gbv_usd','lt_true_up_net_rev_usd')

#Renaming the existing coloumns for better understanding
tempdf = tempdf.withColumnRenamed('local_date', 'activity_date')
tempdf = tempdf.withColumnRenamed('srch_window', 'srch_booking_window')
tempdf = tempdf.withColumnRenamed('xclick', 'traffic')
tempdf = tempdf.withColumnRenamed('xcost', 'cost')
tempdf = tempdf.withColumnRenamed('booked_bkg_window', 'booked_booking_window')
tempdf = tempdf.withColumnRenamed('lt_true_up_gp_usd', 'gross_profit')
tempdf = tempdf.withColumnRenamed('lt_true_up_gbv_usd', 'gross_booking_value')
tempdf = tempdf.withColumnRenamed('lt_true_up_net_rev_usd', 'net_revenue')

In [5]:
#writing the user-defined functions for binning/bucketing

from test_helper import Test

def searchToCategory(srch_los):
    if srch_los == 1: return '1'
    elif srch_los == 2: return'2'
    elif srch_los ==3: return '3'
    elif srch_los ==4: return '4'
    elif srch_los ==5: return '5'
    elif srch_los ==6: return '6'
    elif srch_los ==7: return '7'
    elif srch_los >= 8 and srch_los <= 14: return '8-14'

    else: return '32'
    
    
##testing for function searchToCategory.
Test.assertEquals(searchToCategory(3), '3','incorrect result: not matching')
Test.assertEquals(searchToCategory(9), '8-14','incorrect result: not matching')

    
def searchbucketToCategory(srch_booking_window):
    if srch_booking_window== 1: return '1'
    elif srch_booking_window == 2: return '2'
    elif srch_booking_window >= 3 and srch_booking_window <= 4: return '3-4'
    elif srch_booking_window >= 5 and srch_booking_window <= 7: return '5-7'
    elif srch_booking_window >= 8 and srch_booking_window <= 14: return '8-14'
    elif srch_booking_window >= 15 and srch_booking_window <= 20: return '15-20'
    elif srch_booking_window >= 21 and srch_booking_window <= 30: return '21-30'
    elif srch_booking_window >= 31 and srch_booking_window <= 40: return '31-40'
    elif srch_booking_window >= 41 and srch_booking_window <= 60: return '41-60'
    elif srch_booking_window >= 61 and srch_booking_window <= 90: return '61-90'
    elif srch_booking_window >= 91 and srch_booking_window <= 120: return '91-120'
    elif srch_booking_window >= 121 and srch_booking_window <= 180: return '121-180'
    elif srch_booking_window >= 181 and srch_booking_window <= 240: return '181-240'

    else: return '>240'
    
    
    
    
##testing for function searchbucketToCategory.
Test.assertEquals(searchbucketToCategory(3), '3-4','incorrect result: not matching')
Test.assertEquals(searchbucketToCategory(120), '91-120','incorrect result: not matching')
Test.assertEquals(searchbucketToCategory(250), '>240','incorrect result: not matching')


    
def srchbooked_los_bucket(booked_los):
    if booked_los == 1: return '1'
    elif booked_los == 2: return '2'
    elif booked_los == 3: return '3'
    elif booked_los == 4: return '4'
    elif booked_los == 5: return '5'
    elif booked_los == 6: return '6'
    elif booked_los == 7: return '7'
    elif booked_los >= 8 and booked_los <= 14: return '8-14'
    else: return '>14'
    
    
##testing for function searchbucketToCategory.
Test.assertEquals(srchbooked_los_bucket(3), '3','incorrect result: not matching')
Test.assertEquals(srchbooked_los_bucket(9), '8-14','incorrect result: not matching')


    
def srchbooked_bw_bucket(booked_booking_window):
    if booked_bkg_window == 1: return '1'
    elif booked_bkg_window == 2: return '2'
    elif booked_bkg_window >= 3 or booked_bkg_window <= 4: return '3-4'
    elif booked_bkg_window >= 5 or booked_bkg_window <= 7: return '5-7'
    elif booked_bkg_window >= 8 or booked_bkg_window <= 14: return '8-14'
    elif booked_bkg_window >= 21 or booked_bkg_window <= 30: return '21-30'
    elif booked_bkg_window >= 31 or booked_bkg_window <= 40: return '31-40'
    elif booked_bkg_window >= 41 or booked_bkg_window <= 60: return '41-60'
    elif booked_bkg_window >= 61 or booked_bkg_window <= 90: return '61-90'
    elif booked_bkg_window >= 91 or booked_bkg_window <= 120: return '91-120'
    elif booked_bkg_window >= 121 or booked_bkg_window <= 180: return '121-180'
    elif booked_bkg_window >= 181 or booked_bkg_window <= 240: return '181-240'
    else: return '>240'
    
##testing for function srchbooked_bw_bucket.
#booked_booking_window = 25
#Test.assertEquals(srchbooked_bw_bucket(1), '1','incorrect result: not matching')
#Test.assertEquals(srchbooked_bw_bucket(243), '>240','incorrect result: not matching')




1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.


In [6]:
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType

from pyspark.sql.functions import udf

#registering functions to UDF
udfsearchToCategory=udf(searchToCategory, StringType())
udfsearchbucketToCategory=udf(searchbucketToCategory, StringType())
udfsrchbooked_los_bucket=udf(srchbooked_los_bucket, StringType())
udfsrchsrchbooked_bw_bucket=udf(srchbooked_bw_bucket, StringType())

# Adding UDF's as coloumns to existing dataframe.

tempdf = tempdf.withColumn("srch_los_bucket", udfsearchToCategory("srch_los"))
tempdf = tempdf.withColumn("srch_bw_bucket", udfsearchbucketToCategory("srch_booking_window"))
tempdf = tempdf.withColumn("booked_los_bucket", udfsearchbucketToCategory("booked_los"))
tempdf = tempdf.withColumn("booked_bw_bucket", udfsearchbucketToCategory("booked_booking_window"))

#registering dataframe as temp tabel
tempdf.registerTempTable("new_ytd_feb14to23jun15_temp")


In [7]:
tempc1df = tempdf #Assign existing dataframe to another dataframes to do further operations on it.

# function for checking flag on coloumn.
def booking_value_temp1(booked_flag):
    if booked_flag == 'Y': return '1'
    else: return '0'
    
##Testing for function booking_value_temp1.
Test.assertEquals(booking_value_temp1("Y"),"1",'incorrect result: not matching')
#working


#filter the outliers and selecting the required data
tempc1df = tempc1df.filter((tempc1df['booked_flag'] == 'Y') & ((tempc1df['srch_Booking_window'] < -1) | (tempc1df['srch_los'] <= 0)))

temp1df = tempc1df.select('activity_date','partner_org','partner_pos','device_type','booked_hotel_id','Booked_Booking_window', \
                          'Booked_BW_bucket','Booked_LOS','Booked_LOS_Bucket','traffic','cost','booked_flag','gross_profit', \
                          'gross_booking_value','net_revenue')



#Renaming the existing coloumns for better understanding
temp1df = temp1df.withColumnRenamed('Booked_Booking_window', 'booking_window')
temp1df = temp1df.withColumnRenamed('Booked_BW_bucket', 'bw_bucket')
temp1df = temp1df.withColumnRenamed('Booked_LOS_Bucket', 'los_bucket')
temp1df = temp1df.withColumnRenamed('booked_hotel_id', 'hotel_id')



#registering function to UDF and appending as a coloumn to existing dataframe.
udfbooking_value_temp1=udf(booking_value_temp1, StringType())
temp1df = temp1df.withColumn("booking", udfbooking_value_temp1("booked_flag"))




#registering dataframe as temp tabel
temp1df.registerTempTable("new_ytd_feb14to23jun15_temp1")


1 test passed.


In [8]:
#temp2 tabel

from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType
from pyspark.sql.functions import udf


# function for checking flag on coloumn.
def booking_value_temp2(booked_flag):
    if booked_flag == 'N': return '0'
    else: return '1'
    
from test_helper import Test   
##Testing for function booking_value_temp2
Test.assertEquals(booking_value_temp2("N"),"0",'incorrect result: not matching')




tempc2df = tempdf #Assign existing dataframe to another dataframes to do further operations on it.

#filter the outliers and selecting the required data
tempc2df = tempc2df.filter((tempc1df['booked_flag'] == 'N') & (tempc1df['srch_booking_window'] >= -1) & \
                           (tempc1df['srch_los'] > 0) & (tempc1df['gross_profit'] == 0.0) & (tempc1df['gross_booking_value'] == 0.0))

temp2df = tempc2df.select ('activity_date','partner_org','partner_pos','device_type','clicked_hotel_id', \
                           'srch_booking_window','srch_bw_bucket','srch_LOS','Srch_LOS_Bucket','traffic', \
                           'cost','booked_flag','gross_profit','gross_booking_value','Net_revenue')


#Renaming the existing coloumns for better understanding
temp2df = temp2df.withColumnRenamed('clicked_hotel_id', 'hotel_id')
temp2df = temp2df.withColumnRenamed('srch_booking_window', 'booking_window')
temp2df = temp2df.withColumnRenamed('srch_bw_bucket', 'bw_bucket')
temp2df = temp2df.withColumnRenamed('srch_LOS', 'los')
temp2df = temp2df.withColumnRenamed('Srch_LOS_Bucket', 'los_bukcet')


#registering function to UDF and appending as a coloumn to existing dataframe.
udfbooking_value_temp2=udf(booking_value_temp2, StringType())
temp2df = temp2df.withColumn("booking", udfbooking_value_temp2("booked_flag"))


#registering dataframe as temp tabel
temp2df.registerTempTable("new_ytd_feb14to23jun15_temp2")




1 test passed.


In [9]:
#temp3 table

from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType
from pyspark.sql.functions import udf



# function for checking flag on coloumn.
def booking_value_temp3(booked_flag):
    if booked_flag == 'Y': return '1'
    else: return '0'
    
from test_helper import Test   
##Testing for function booking_value_temp2
Test.assertEquals(booking_value_temp3("Y"),"1",'incorrect result: not matching')


tempc3df = tempdf #Assign existing dataframe to another dataframes to do further operations on it.

#filter the outliers and selecting the required data
tempc3df = tempc3df.filter((tempc1df['booked_flag'] == 'Y') & (tempc1df['srch_booking_window'] >= -1) & \
                           (tempc1df['srch_los'] > 0))
temp3df = tempc3df.select ('activity_date','partner_org','partner_pos','device_type','clicked_hotel_id', \
                           'srch_booking_window','srch_bw_bucket','srch_LOS','Srch_LOS_Bucket','traffic', \
                           'cost','booked_flag','gross_profit','gross_booking_value','Net_revenue')

#Renaming the existing coloumns for better understanding
temp3df = temp3df.withColumnRenamed('clicked_hotel_id', 'hotel_id')
temp3df = temp3df.withColumnRenamed('srch_booking_window', 'booking_window')
temp3df = temp3df.withColumnRenamed('srch_bw_bucket', 'bw_bucket')
temp3df = temp3df.withColumnRenamed('srch_LOS', 'los')
temp3df = temp3df.withColumnRenamed('Srch_LOS_Bucket', 'los_bukcet')


#registering function to UDF and appending as a coloumn to existing dataframe.
udfbooking_value_temp3=udf(booking_value_temp3, StringType())
temp3df = temp3df.withColumn("booking", udfbooking_value_temp3("booked_flag"))


#registering dataframe as temp tabel
temp3df.registerTempTable("new_ytd_feb14to23jun15_temp3")





1 test passed.


In [10]:

#union of all three temporary tables

unionalldf = sqlContext.sql("select *  from new_ytd_feb14to23jun15_temp1 union all select * from new_ytd_feb14to23jun15_temp2 union all \
                             select * from new_ytd_feb14to23jun15_temp3")

#registering to a temporary tabel.
unionalldf.registerTempTable("new_ytd_feb14to23jun15_uninoall")



In [ ]:
## union of temp1,temp2,temp3 tables completed,now going to work on hotel dimensions and country region tables.

In [16]:
path = "/home/affine/vishnu/Inputs/Hotel_Dim.csv" #path of hotel dimenssion tabel data.

In [17]:
#loading hotel dimension tabel data and creating hotel dimension dataframe
hotel_masterdf = sqlCtx.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',delimiter=',').load(path) 


In [18]:

# creating dataframe subset from original hotel dimension dataframe.

hoteltempdf = hotel_masterdf.select("hotel_id","city","region","market","brand","star_rating","hotel_type","trvlr_recmnd_cnt")

hoteldf = hoteltempdf.filter((hoteltempdf['hotel_id'].isNotNull()))

hoteldf.registerTempTable("new_ytd_feb14to23jun15_hotel_dim")# registering dataframe as temporary tabel.

In [19]:
# creating ad_canada tabel

ad_canadadf = sqlContext.sql("select new_ytd_feb14to23jun15_uninoall.*,new_ytd_feb14to23jun15_hotel_dim.hotel_id ,city, \
                         region,new_ytd_feb14to23jun15_hotel_dim.market,new_ytd_feb14to23jun15_hotel_dim.star_rating, \
                         new_ytd_feb14to23jun15_hotel_dim.brand,new_ytd_feb14to23jun15_hotel_dim.trvlr_recmnd_cnt,new_ytd_feb14to23jun15_hotel_dim.hotel_type from new_ytd_feb14to23jun15_uninoall left join new_ytd_feb14to23jun15_hotel_dim on new_ytd_feb14to23jun15_uninoall.hotel_id == new_ytd_feb14to23jun15_hotel_dim.hotel_id")

# registering dataframe as temporary tabel.
ad_canadadf.registerTempTable("new_ytd_feb14to23jun15_ad_canada")





In [20]:
# canada region table

from pyspark.sql.functions import lit

#filter the outliers and selecting the required data
canada_df = ad_canadadf.filter(ad_canadadf["region"] =='CANADA')

#writing the user-defined functions for binning/bucketing

def srchbcanada_region_city(city): 
    if city == 'TORONTO': return 'TORONTO'
    elif city == 'NIAGARA FALLS': return 'NIAGARA FALLS'
    elif city == 'MONTREAL': return 'MONTREAL'
    elif city == 'VANCOUVER': return 'VANCOUVER'
    elif city == 'QUEBEC': return 'QUEBEC'
    elif city == 'BANFF': return 'BANFF'
    elif city == 'OTTAWA': return 'OTTAWA'
    elif city == 'EDMONTON': return 'EDMONTON'
    elif city == 'VICTORIA': return 'VICTORIA'
    elif city == 'CALGARY': return 'CALGARY'
    elif city == 'WHISTLER': return 'WHISTLER'
    elif city == 'NIAGARA-ON-THE-LAKE' or 'CANMORE' or 'JASPER' or 'KELOWNA' or 'HALIFAX' or 'MISSISSAUGA' or  \
    'LAKE LOUISE' or 'COLLINGWOOD' or 'VERNON' or 'PENTICTON' or 'GATINEAU' or 'DORVAL' or 'MARKHAM': return 'G12'
    elif city == 'MONT-TREMBLANT' or 'TOFINO' or 'BLUE MOUNTAINS' or 'OSOYOOS' or 'UCLUELET' or 'SAINT-SAUVEUR' or \
    'WEST KELOWNA' or 'LA MALBAIE' or 'SAINTE-ADELE': return 'G13'
    elif city == 'WINNIPEG' or 'LONDON' or 'RICHMOND' or 'BARRIE' or 'REVELSTOKE' or 'VAUGHAN' or 'ORILLIA' or  \
    'BURNABY' or 'SURREY': return 'G14'
    else : 'others'
        
        
##testing for function srchbcanada_region_city.
Test.assertEquals(srchbcanada_region_city("TORONTO"), "TORONTO",'incorrect result: not matching')
Test.assertEquals(srchbcanada_region_city("NIAGARA-ON-THE-LAKE"), "G12",'incorrect result: not matching')
   

        
def srchbcanada_region_star_rating(star_rating):
    if star_rating == '2.0': return '2'
    elif star_rating == '2.5': return '2.5'
    elif star_rating == '3.0': return '3'
    elif star_rating == '3.5': return '3.5'
    elif star_rating == '4.0': return '4'
    elif star_rating == '1.0' or star_rating == '1.5': return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0': return '4.5-5'
    else : '0'
        
        
##testing for function srchbcanada_region_star_rating.
Test.assertEquals(srchbcanada_region_star_rating('2.5'), '2.5','incorrect result: not matching')
Test.assertEquals(srchbcanada_region_star_rating('1.5'), '1-1.5','incorrect result: not matching')



        
        
def srchbcanada_region_Hotel_type(Hotel_type):
    if Hotel_type == 'HOTEL': return 'HOTEL'
    elif Hotel_type == 'HOTEL RESORT': return 'HOTEL RESORT'
    elif Hotel_type == 'CONDOMINIUM RESORT' and Hotel_type == 'CONDO': return 'CONDO'
    elif Hotel_type == 'LODGE' or Hotel_type == 'BED & BREAKFAST' or Hotel_type == 'HOSTEL/BACKPACKER ACCOMMODATION' or \
      'CABIN' or Hotel_type == 'GUEST HOUSE': return 'Budget_hotel'
    elif Hotel_type == 'APART-HOTEL' and Hotel_type == 'INN': return 'Family_Premium'
    else : 'Others'
        
##testing for function srchbcanada_region_star_rating.
Test.assertEquals(srchbcanada_region_Hotel_type('HOTEL'), 'HOTEL','incorrect result: not matching')

        
def srchbcanada_region_market(market):
    if market == 'TORONTO, ON, CAN': return 'TORONTO, ON, CAN'
    elif market == 'NIAGARA FALLS, ON, CAN': return 'NIAGARA FALLS, ON, CAN'
    elif market == 'MONTREAL, QC, CAN' : return 'MONTREAL, QC, CAN'
    elif market == 'BANFF AREA, AB, CAN' : return 'BANFF AREA, AB, CAN'
    elif market == 'VANCOUVER, BC, CAN' : return 'VANCOUVER, BC, CAN'
    elif market == 'QUEBEC CITY, QC, CAN' : return 'QUEBEC CITY, QC, CAN'
    elif market == 'EDMONTON, AB, CAN' : return 'EDMONTON, AB, CAN'
    elif market == 'OTTAWA, ON, CAN' : return 'OTTAWA, ON, CAN'
    elif market == 'OKANAGAN VALLEY, BC, CAN' : return 'OKANAGAN VALLEY, BC, CAN'
    elif market == 'VICTORIA, BC, CAN' or market == 'CALGARY, AB, CAN'or market == 'JASPER AREA, AB, CAN' : return 'G10'
    elif market == 'WHISTLER, BC, CAN' or market == 'NIAGARA-ON-THE-LAKE, ON, CAN' or market == 'MONT TREMBLANT, QC, CAN' or market == 'UCLUELET, BC, CAN': return 'G11'
    elif market == 'COLLINGWOOD, ON, CAN' or market == 'BC INTERIOR SKI, BC, CAN' or market == 'HALIFAX, NS, CAN' or market == 'WINNIPEG, MB, CAN' or \
    market == 'LONDON, ON, CAN' or market == 'KAMLOOPS, BC, CAN': return 'G12'   
    else : 'others' 
        
        
##testing for function srchbcanada_region_star_rating.
Test.assertEquals(srchbcanada_region_market('TORONTO, ON, CAN'), 'TORONTO, ON, CAN','incorrect result: not matching')
Test.assertEquals(srchbcanada_region_market('COLLINGWOOD, ON, CAN'), 'G12','incorrect result: not matching')

        
#Registering functions as UDF
udfsrchbcanada_region_city=udf(srchbcanada_region_city, StringType())
udfsrchbcanada_region_star_rating=udf(srchbcanada_region_star_rating, StringType())
udfsrchbcanada_region_Hotel_type=udf(srchbcanada_region_Hotel_type, StringType())
udfsrchbcanada_region_market=udf(srchbcanada_region_market, StringType())


#adding udf's to existing dataframe as coloumns.
canada_df = canada_df.withColumn("city_bucket", udfsrchbcanada_region_city("city"))
canada_df = canada_df.withColumn("star_rating_bucket", udfsrchbcanada_region_star_rating("star_rating"))
canada_df = canada_df.withColumn("hotel_type_bucket", udfsrchbcanada_region_Hotel_type("Hotel_type"))
canada_df = canada_df.withColumn("market_bucket", udfsrchbcanada_region_market("market"))
canada_df = canada_df.withColumn("region_bucket", lit("NA"))
canada_df = canada_df.withColumn("model_name", lit("Canada"))

# registering dataframe as temporary tabel.
canada_df.registerTempTable("new_ytd_feb14to23jun15_canada")



1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.


In [21]:

# US region tabel


#filter the outliers and selecting the required data
usdf = ad_canadadf.filter((ad_canadadf["region"] == 'FLORIDA') |(ad_canadadf["region"] == 'NEW YORK CITY') \
                          |(ad_canadadf["region"] == 'CALIFORNIA') |(ad_canadadf["region"] == 'GAMING') \
                          |(ad_canadadf["region"] == 'NORTHEAST') |(ad_canadadf["region"] == 'MOUNTAIN') \
                          | (ad_canadadf["region"] == 'MIDWEST') |(ad_canadadf["region"] == 'CENTRAL') \
                          | (ad_canadadf["region"] == 'SF PACNW') |(ad_canadadf["region"] == 'ORLANDO') \
                          | (ad_canadadf["region"] == 'SF PACNW')) 


#writing the user-defined functions for binning/bucketing
def srchbUS_region_star_rating(star_rating):
    if star_rating == '2.0': return '2'
    elif star_rating == '2.5': return '2.5'
    elif star_rating == '3.0': return '3'
    elif star_rating == '3.5': return '3.5'
    elif star_rating == '4.0': return '4'
    elif star_rating == '1.0' or star_rating == '1.5': return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0': return '4.5-5'
    else : '0'
        
#testing for function srchbcanada_region_city
Test.assertEquals(srchbUS_region_star_rating('2.0'), '2','incorrect result: not matching')
Test.assertEquals(srchbUS_region_star_rating('1.0'), '1-1.5','incorrect result: not matching')
        
def srchbUS_region_Hotel_type(Hotel_type):
    if Hotel_type == 'HOTEL': return 'HOTEL'
    elif Hotel_type == 'HOTEL RESORT': return 'HOTEL RESORT'
    elif Hotel_type == 'CONDOMINIUM RESORT' or Hotel_type == 'CONDO': return 'CONDO'
    elif Hotel_type == 'LODGE' or Hotel_type == 'BED & BREAKFAST' or Hotel_type == 'HOSTEL/BACKPACKER ACCOMMODATION' or \
      'CABIN' or Hotel_type == 'GUEST HOUSE': return 'Budget_hotel'
    elif Hotel_type == 'APART-HOTEL' or Hotel_type == 'INN': return 'Family_Premium'
    else : 'Others'
        
#testing for function srchbcanada_region_city
Test.assertEquals(srchbUS_region_Hotel_type('HOTEL'), 'HOTEL','incorrect result: not matching')
Test.assertEquals(srchbUS_region_Hotel_type('GUEST HOUSE'), 'Budget_hotel','incorrect result: not matching')


#registering functions as UDF
udfsrchbUS_region_star_rating=udf(srchbUS_region_star_rating, StringType())
udfsrchbUS_region_Hotel_type=udf(srchbUS_region_Hotel_type, StringType())

#adding udf's to existing dataframe as coloumns.
usdf = usdf.withColumn("star_rating_bucket", udfsrchbUS_region_star_rating("star_rating"))
usdf = usdf.withColumn("hotel_type_bucket", udfsrchbUS_region_Hotel_type("Hotel_type"))
usdf = usdf.withColumn("city_bucket", lit("NA"))
usdf = usdf.withColumn("market_bucket", lit("NA"))
usdf = usdf.withColumn("region_bucket", lit("NA"))
usdf = usdf.withColumn("model_name", lit("US"))


# registering dataframe as temporary tabel.
usdf.registerTempTable("new_ytd_feb14to23jun15_US")






1 test passed.
1 test passed.
1 test passed.
1 test passed.


In [ ]:
##Mexico region tabel


#filter the outliers and selecting the required data
mexicodf = ad_canadadf.filter(ad_canadadf["region"] == "MEXICO & CENTRAL AMERICA")


#writing the user-defined functions for binning/bucketing
def srchbMexico_region_star_rating(star_rating):
    if star_rating == '2.0': return '2'
    elif star_rating == '2.5': return '2.5'
    elif star_rating == '3.0': return '3'
    elif star_rating == '3.5': return '3.5'
    elif star_rating == '4.0': return '4'
    elif star_rating == '1.0' or star_rating == '1.5': return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0': return '4.5-5'
    else : '0'
        
#testing for function srchbcanada_region_city
Test.assertEquals(srchbMexico_region_star_rating('2.0'), '2','incorrect result: not matching')
Test.assertEquals(srchbMexico_region_star_rating('1.0'), '1-1.5','incorrect result: not matching')
 

        
def srchbMexico_region_Hotel_type(Hotel_type):
    if Hotel_type == 'HOTEL': return 'HOTEL'
    elif Hotel_type == 'HOTEL RESORT': return 'HOTEL RESORT'
    elif Hotel_type == 'ALL-INCLUSIVE' : return 'ALL-INCLUSIVE'
    elif Hotel_type == 'CONDOMINIUM RESORT' or Hotel_type == 'CONDO': return 'CONDO'
    elif Hotel_type == 'LODGE' or Hotel_type == 'BED & BREAKFAST' or Hotel_type == 'HOSTEL/BACKPACKER ACCOMMODATION' or \
      'CABIN' or Hotel_type == 'GUEST HOUSE': return 'Budget_hotel'
    elif Hotel_type == 'APART-HOTEL' or Hotel_type == 'INN': return 'Family_Premium'
    else : 'Others'
        
#testing for function srchbcanada_region_city
Test.assertEquals(srchbMexico_region_Hotel_type('HOTEL'), 'HOTEL','incorrect result: not matching')
Test.assertEquals(srchbMexico_region_Hotel_type('GUEST HOUSE'), 'Budget_hotel','incorrect result: not matching')
        
    
def srchbMexico_region_market(market):
    if market == 'MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM': return 'MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM'
    elif market == 'MEXICO - CANCUN & ISLA MUJERES': return 'MEXICO - CANCUN & ISLA MUJERES'
    elif market == 'MEXICO - PUERTO VALLARTA, NUEVO VALLARTA & RIVIERA NAYARIT' : return 'MEXICO - PUERTO VALLARTA, NUEVO VALLARTA & RIVIERA NAYARIT'
    elif market == 'MEXICO - LOS CABOS' : return 'MEXICO - LOS CABOS'
    elif market == 'CENTRAL AMERICA - COSTA RICA - GUANACASTE' : return 'CENTRAL AMERICA - COSTA RICA - GUANACASTE'
    elif market == 'CENTRAL AMERICA - BELIZE' : return 'CENTRAL AMERICA - BELIZE'
    elif market == 'CENTRAL AMERICA - COSTA RICA - PUNTARENAS' : return 'CENTRAL AMERICA - COSTA RICA - PUNTARENAS'
    elif market == 'CENTRAL AMERICA - COSTA RICA - OTHER' : return 'CENTRAL AMERICA - COSTA RICA - OTHER'
    elif market == 'MEXICO - COZUMEL' : return 'MEXICO - COZUMEL'
    elif market == 'MEXICO - HUATULCO' : return 'MEXICO - HUATULCO'
    elif market == 'MEXICO - IXTAPA & ZIHUATANEJO' : return 'MEXICO - IXTAPA & ZIHUATANEJO'
    elif market == 'CENTRAL AMERICA - HONDURAS' : return 'CENTRAL AMERICA - HONDURAS'
    elif market == 'CENTRAL AMERICA - PANAMA - OTHER' : return 'CENTRAL AMERICA - PANAMA - OTHER'
    elif market == 'CENTRAL AMERICA - COSTA RICA - SAN JOSE' or market == 'CENTRAL AMERICA - PANAMA - PANAMA CITY' or \
         market == 'MEXICO - MEXICO CITY' or market == 'MEXICO - MEXICO CITY' : return 'hclt'
    elif market == 'CENTRAL AMERICA - NICARAGUA' or market == 'MEXICO - MAZATLAN' or \
         market == 'MEXICO - MANZANILLO' or market == 'CENTRAL AMERICA - EL SALVADOR'  or market == 'MEXICO - ACAPULCO': return 'hclt'
      
    else : 'Low_traffic'  
        
#testing for function srchbcanada_region_city
Test.assertEquals(srchbMexico_region_market('MEXICO - CANCUN & ISLA MUJERES'), 'MEXICO - CANCUN & ISLA MUJERES','incorrect result: not matching')
Test.assertEquals(srchbMexico_region_market('MEXICO - MANZANILLO'), 'hclt','incorrect result: not matching')
        
        
#Registering functions as UDF's        
udfsrchbMexico_region_star_rating=udf(srchbMexico_region_star_rating, StringType())
udfsrchbMexico_region_Hotel_type=udf(srchbMexico_region_Hotel_type, StringType())
udfsrchbMexico_region_market=udf(srchbMexico_region_market, StringType())

#Adding the udf as coloumns to existing data frame.
mexicodf = mexicodf.withColumn("star_rating_bucket", udfsrchbMexico_region_star_rating("star_rating"))
mexicodf = mexicodf.withColumn("hotel_type_bucket", udfsrchbMexico_region_Hotel_type("Hotel_type"))
mexicodf = mexicodf.withColumn("market_bucket", udfsrchbMexico_region_market("market"))
mexicodf = mexicodf.withColumn("city_bucket", lit("NA"))
mexicodf = mexicodf.withColumn("region_bucket", lit("NA"))
mexicodf = mexicodf.withColumn("model_name", lit("Mexico"))
        


# registering dataframe as temporary tabel.    
mexicodf.registerTempTable("new_ytd_feb14to23jun15_Mexico")
    



In [ ]:
## Caribbean region tabel

In [ ]:

#filter the outliers and selecting the required data
Caribbeandf = ad_canadadf.filter(ad_canadadf["region"] == "CARIBBEAN")

#writing the user-defined functions for binning/bucketing
def srchbCaribbean_region_star_rating(star_rating):
    if star_rating == '2.0': return '2'
    elif star_rating == '2.5': return '2.5'
    elif star_rating == '3.0': return '3'
    elif star_rating == '3.5': return '3.5'
    elif star_rating == '4.0': return '4'
    elif star_rating == '1.0' or star_rating == '1.5': return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0': return '4.5-5'
    else : '0'
        
#testing for function srchbcanada_region_city
Test.assertEquals(srchbCaribbean_region_star_rating('4.0'), '4','incorrect result: not matching')
Test.assertEquals(srchbCaribbean_region_star_rating('4.5'), '4.5-5','incorrect result: not matching')        

        
        
def srchbCaribbean_region_Hotel_type(Hotel_type):
    if Hotel_type == 'HOTEL': return 'HOTEL'
    elif Hotel_type == 'HOTEL RESORT': return 'HOTEL RESORT'
    elif Hotel_type == 'ALL-INCLUSIVE' : return 'ALL-INCLUSIVE'
    elif Hotel_type == 'CONDOMINIUM RESORT' or Hotel_type == 'CONDO': return 'CONDO'
    elif Hotel_type == 'APART-HOTEL' or Hotel_type == 'INN': return 'Family_Premium'
    else : 'Others'
        
#testing for function srchbcanada_region_city
Test.assertEquals(srchbCaribbean_region_Hotel_type('HOTEL'), 'HOTEL','incorrect result: not matching')
Test.assertEquals(srchbCaribbean_region_Hotel_type('APART-HOTEL'), 'Family_Premium','incorrect result: not matching')  
        
        
def srchbCaribbean_region_market(market):
    if market == 'PUNTA CANA, DOMINICAN REPUBLIC': return 'PUNTA CANA, DOMINICAN REPUBLIC'
    elif market == 'TURKS AND CAICOS': return 'TURKS AND CAICOS'
    elif market == 'MONTEGO BAY, JAMAICA' : return 'MONTEGO BAY, JAMAICA'
    elif market == 'ST. LUCIA' : return 'ST. LUCIA'
    elif market == 'OCHO RIOS, JAMAICA' : return 'OCHO RIOS, JAMAICA'
    elif market == 'NEGRIL, JAMAICA' : return 'NEGRIL, JAMAICA'
    elif market == 'BARBADOS' : return 'BARBADOS'
    elif market == 'ARUBA' or market == 'PUERTO PLATA, DOMINICAN REPUBLIC' or market == 'ANTIGUA AND BARBUDA' : return 'G8'
    elif market == 'NASSAU, BAHAMAS' or market == 'DOMINICAN REPUBLIC ALL OTHER' or \
         market == 'BERMUDA' or market == 'BAHAMAS, OUT ISLANDS' : return 'G9'
    elif market == 'SAN JUAN, PUERTO RICO' or market == 'PARADISE ISLAND, BAHAMAS' or market == 'CAYMAN ISLANDS' or \
          market == 'SINT MAARTEN, DUTCH'  or market == 'CURACAO' or market == 'ST. MARTIN, FRENCH': return 'G10'  
    else : 'Others'  
        
        
#testing for function srchbcanada_region_city
Test.assertEquals(srchbCaribbean_region_market('TURKS AND CAICOS'), 'TURKS AND CAICOS','incorrect result: not matching')
Test.assertEquals(srchbCaribbean_region_market('SINT MAARTEN, DUTCH'), 'G10','incorrect result: not matching')
        
#Registering functions to UDF's
udfsrchbCaribbean_region_star_rating=udf(srchbCaribbean_region_star_rating, StringType())
udfsrchbCaribbean_region_Hotel_type=udf(srchbCaribbean_region_Hotel_type, StringType())
udfsrchbCaribbean_region_market=udf(srchbCaribbean_region_market, StringType())

#adding udf as coloumn to existing dataframe
Caribbeandf = Caribbeandf.withColumn("star_rating_bucket", udfsrchbCaribbean_region_star_rating("star_rating"))
Caribbeandf = Caribbeandf.withColumn("hotel_type_bucket", udfsrchbCaribbean_region_Hotel_type("Hotel_type"))
Caribbeandf = Caribbeandf.withColumn("market_bucket", udfsrchbCaribbean_region_market("market"))
Caribbeandf = Caribbeandf.withColumn("city_bucket",lit("NA"))
Caribbeandf = Caribbeandf.withColumn("region_bucket",lit("NA"))
Caribbeandf = Caribbeandf.withColumn("model_name",lit("CARIBBEAN"))


# registering dataframe as temporary tabel.
Caribbeandf.registerTempTable("new_ytd_feb14to23jun15_Caribbean")





In [ ]:
#other region table

In [ ]:
#filter the outliers and selecting the required data
other_regiondf = ad_canadadf.filter((ad_canadadf["region"] == 'SPAIN & PORTUGAL') |(ad_canadadf["region"] == 'ROME & ITALY RESORTS')\
                                    |(ad_canadadf["region"] == 'MID-ATLANTIC') |(ad_canadadf["region"] == 'GREECE & TURKEY') \
                                   |(ad_canadadf["region"] == 'PARIS') |(ad_canadadf["region"] == 'OCEANIA') \
                                   | (ad_canadadf["region"] == 'ITALY NORTH') |(ad_canadadf["region"] == 'LONDON, ENGLAND') \
                                   | (ad_canadadf["region"] == 'THAILAND') |(ad_canadadf["region"] == 'MIDDLE EAST AND INDIAN OCEAN') \
                                   | (ad_canadadf["region"] == 'EUROPE REGIONAL TERRITORIES') |(ad_canadadf["region"] == 'UK & IRELAND') \
                                   | (ad_canadadf["region"] == 'AFRICA AND EAST MED') |(ad_canadadf["region"] == 'EASTERN EUROPE') \
                                   | (ad_canadadf["region"] == 'GERMANY, AUSTRIA & SWITZERLAND') |(ad_canadadf["region"] == 'JAPAN & MICRONESIA') \
                                   | (ad_canadadf["region"] == 'PHILIPPINES') |(ad_canadadf["region"] == 'BENELUX') |(ad_canadadf["region"] == 'FRANCE') \
                                   | (ad_canadadf["region"] == 'INDONESIA') |(ad_canadadf["region"] == 'INDIAN SUBCONTINENT') |(ad_canadadf["region"] == 'HONG KONG & MACAU') \
                                   | (ad_canadadf["region"] == 'SOUTHEAST ASIA EMERGING') |(ad_canadadf["region"] == 'NORDIC') \
                                   | (ad_canadadf["region"] == 'MAINLAND CHINA') |(ad_canadadf["region"] == 'SOUTH KOREA')  |(ad_canadadf["region"] == 'SINGAPORE (REGION)') \
                                   | (ad_canadadf["region"] == 'MALAYSIA') |(ad_canadadf["region"] == 'TAIWAN') |(ad_canadadf["region"] == 'UNKNOWN') \
                                   | (ad_canadadf["region"] == 'SOUTH AMERICA'))

#writing the user-defined functions for binning/bucketing
def srchbother_region_star_rating(star_rating):
    if star_rating == '2.0': return '2'
    elif star_rating == '2.5': return '2.5'
    elif star_rating == '3.0': return '3'
    elif star_rating == '3.5': return '3.5'
    elif star_rating == '4.0': return '4'
    elif star_rating == '1.0' or star_rating == '1.5': return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0': return '4.5-5'
    else : '0'
        
#testing for function srchbother_region_star_rating
Test.assertEquals(srchbother_region_star_rating('2.0'), '2','incorrect result: not matching')
Test.assertEquals(srchbother_region_star_rating('4.5'), '4.5-5','incorrect result: not matching')
        
        
def srchbother_region_Hotel_type(Hotel_type):
    if Hotel_type == 'HOTEL': return 'HOTEL'
    elif Hotel_type == 'HOTEL RESORT': return 'HOTEL RESORT'
    elif Hotel_type == 'APART-HOTEL' : return 'APART-HOTEL'
    elif Hotel_type == 'GUEST HOUSE' or Hotel_type == 'BED & BREAKFAST' or Hotel_type == 'HOSTEL/BACKPACKER ACCOMMODATION'  \
     or Hotel_type == 'MOTEL'or Hotel_type == 'HOSTAL': return 'Budget_hotel'
    elif Hotel_type == 'APARTMENT' or Hotel_type == 'RESIDENCE' or Hotel_type == 'TOWNHOUSE'  \
     or Hotel_type == 'INN': return 'Budget_hotel'
    else : 'Others'
        
#testing for function srchbother_region_Hotel_type
Test.assertEquals(srchbother_region_Hotel_type('HOTEL'), 'HOTEL','incorrect result: not matching')
Test.assertEquals(srchbother_region_Hotel_type('MOTEL'), 'Budget_hotel','incorrect result: not matching')
        
        
def srchbother_region_Region_bucket(region):
    if region == 'SPAIN & PORTUGAL': return 'SPAIN & PORTUGAL'
    elif region == 'ROME & ITALY RESORTS': return 'ROME & ITALY RESORTS'
    elif region == 'MID-ATLANTIC': return 'MID-ATLANTIC'
    elif region == 'GREECE & TURKEY': return 'GREECE & TURKEY'
    elif region == 'OCEANIA': return 'OCEANIA'
    elif region == 'ITALY NORTH': return 'ITALY NORTH'
    elif region == 'THAILAND': return 'THAILAND'
    elif region == 'MIDDLE EAST AND INDIAN OCEAN': return 'MIDDLE EAST AND INDIAN OCEAN'
    elif region == 'PARIS' or region == 'LONDON' or region == 'ENGLAND': return 'G9'
    elif region == 'EUROPE REGIONAL TERRITORIES' or region == 'UK & IRELAND' : return 'G10'
    elif region == 'SOUTH AMERICA' or region == 'AFRICA AND EAST MED' : return 'G11'
    elif region == 'EASTERN EUROPE' or region == 'GERMANY, AUSTRIA & SWITZERLAND' or region == 'BENELUX' or region == 'FRANCE' or region == 'NORDIC': return 'G12'
    elif region == 'JAPAN & MICRONESIA' or region == 'HONG KONG & MACAU' or region == 'SOUTHEAST ASIA EMERGING' or region == 'MAINLAND CHINA' : return 'G13'
    else : 'Others'
        
#testing for function srchbother_region_Hotel_type
Test.assertEquals(srchbother_region_Region_bucket('SPAIN & PORTUGAL'), 'SPAIN & PORTUGAL','incorrect result: not matching')
Test.assertEquals(srchbother_region_Region_bucket('JAPAN & MICRONESIA'), 'G13','incorrect result: not matching')


#Registering functions to UDF's   
udfsrchbother_region_star_rating=udf(srchbother_region_star_rating, StringType())
udfsrchbother_region_Hotel_type=udf(srchbother_region_Hotel_type, StringType())
udfsrchbother_region_Region_bucket=udf(srchbother_region_Region_bucket, StringType())

#adding udf as coloumn to existing dataframe
other_regiondf = other_regiondf.withColumn("star_rating_bucket", udfsrchbother_region_star_rating("star_rating"))
other_regiondf = other_regiondf.withColumn("hotel_type_bucket", udfsrchbother_region_Hotel_type("Hotel_type"))
other_regiondf = other_regiondf.withColumn("region_bucket", udfsrchbother_region_Region_bucket("region"))
other_regiondf = other_regiondf.withColumn("city_bucket", lit("NA"))
other_regiondf = other_regiondf.withColumn("model_name", lit("Other Region"))

    
# registering dataframe as temporary tabel.
other_regiondf.registerTempTable("new_ytd_feb14to23jun15_Other_region")
    
    



In [ ]:
# Northamerica region tabel

In [ ]:

#filter the outliers and selecting the required data
NAdf = ad_canadadf.filter(ad_canadadf["region"] == "NORTH AMERICA REGIONAL TERRITORIES")

#writing the user-defined functions for binning/bucketing
def srchbNAdf_region_star_rating(star_rating):
    if star_rating == '2.0': return '2'
    elif star_rating == '2.5': return '2.5'
    elif star_rating == '3.0': return '3'
    elif star_rating == '3.5': return '3.5'
    elif star_rating == '4.0': return '4'
    elif star_rating == '1.0' or star_rating == '1.5': return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0': return '4.5-5'
    else : '0'
        
#testing for function srchbother_region_Hotel_type
Test.assertEquals(srchbNAdf_region_star_rating('2.0'), '2','incorrect result: not matching')
Test.assertEquals(srchbNAdf_region_star_rating('4.5'), '4.5-5','incorrect result: not matching')
        
        
def srchbNA_region_Hotel_type(Hotel_type):
    if Hotel_type == 'HOTEL': return 'HOTEL'
    elif Hotel_type == 'MOTEL': return 'MOTEL'
    elif Hotel_type == 'LODGE' or Hotel_type == 'BED & BREAKFAST' or Hotel_type == 'COTTAGE'  \
     or Hotel_type == 'GUEST HOUSE'or Hotel_type == 'CABIN': return 'Budget_hotel'
    elif Hotel_type == 'APARTMENT' or Hotel_type == 'HOTEL RESORT' or Hotel_type == 'CONDO'  \
     or Hotel_type == 'INN' or Hotel_type == 'APART-HOTEL' or Hotel_type == 'CARAVAN PARK': return 'Family_Premium'
    else : 'Others'
        
#testing for function srchbother_region_Hotel_type
Test.assertEquals(srchbNA_region_Hotel_type('HOTEL'), 'HOTEL','incorrect result: not matching')
Test.assertEquals(srchbNA_region_Hotel_type('INN'), 'Family_Premium','incorrect result: not matching')
       
        
        
def srchbNA_region_market(market):
    if market == 'GRAVENHURST - BRACEBRIDGE - HUNTSVILLE, ON, CAN' or market == 'ONTARIO WEST, ON, CAN' or market == 'ABBOTSFORD - CHILLIWACK, BC, CAN' or market =='MAINE - COAST' \
    or market == 'ONTARIO EAST, ON, CAN' or market == 'VANCOUVER ISLAND, BC, CAN' or market == 'BURLINGTON, VT' \
    or market == 'CHARLOTTETOWN, PEI, CAN': return 'G1'
    
    elif market == 'ALBERTA EAST, AB, CAN' or market == 'REGINA, SK, CAN' or market == 'KINGSTON, ON, CAN' or market == 'SASKATOON, SK, CAN'  \
    or market == 'MONCTON, NB, CAN' or market == 'TRI-CITIES (KW-GUELPH-CAMBRIDGE), ON, CAN': return 'G2'
    
    elif market == 'WINDSOR, ON, CAN' or market == 'RED DEER, AB, CAN' or market == 'BRITISH COLUMBIA SOUTH, BC, CAN' or market == 'GRAND FORKS, ND'  \
    or market == 'SUDBURY, ON, CAN' or market == 'ONTARIO NORTH, ON, CAN'or market == 'BANGOR, ME' \
    or market == 'HAMILTON - BRANTFORD, ON, CAN' or market == 'BELLEVILLE - TRENTON - COBOURG, ON, CAN'or market == 'FARGO, ND': return 'G3'
    
    elif market == 'LAKE PLACID, NY' or market == 'PARKSVILLE, VANCOUVER ISLAND, BC, CAN' or market == 'LAKE GEORGE, NY' or market == 'OGUNQUIT, ME' or market =='EVERETT, WA'  \
    or market == 'COASTAL, BC, CAN' or market == 'GROVE CITY, PA' or market == 'BELLINGHAM, WA' \
    or market == 'NORTH CONWAY, NH' or market == 'NEWFOUNDLAND OTHER, NL, CAN' or market == 'NOVA SCOTIA SOUTH, NS, CAN' \
    or market == 'PORTSMOUTH, NH' : return 'G4'
        
    elif market == 'LETHBRIDGE, AB, CAN' or market == 'FREDERICTON, NB, CAN' or market == 'BRANDON, MB, CAN' or market == 'ERIE, PA'  \
    or market == 'NANAIMO, VANCOUVER ISLAND, BC, CAN' or market == 'SASKATCHEWAN SOUTH, SK, CAN' or market == 'PRINCE GEORGE - QUESNEL, BC, CAN' \
    or market == 'MEDICINE HAT, AB, CAN' or market == 'GRANDE PRAIRIE, AB, CAN' or market == 'ALBERTA NORTH, AB, CAN' \
    or market == 'DURHAM, ON, CAN' or market == 'BRITISH COLUMBIA CENTRAL, BC, CAN' or market == 'ONTARIO SOUTHWEST, ON, CAN' \
    or market == 'SAULT STE. MARIE, ON, CAN' or market == 'THUNDER BAY, ON, CAN' or market == 'SAINT JOHN, NB, CAN' \
    or market == 'GREAT FALLS, MT' or market == 'MINOT, ND (AREA)' or market == 'BRITISH COLUMBIA NORTH, BC, CAN' \
    or market == 'NORTH BAY, ON, CAN' or market == 'PETERBOROUGH, ON, CAN' or market == 'CAPE BRETON ISLAND, NS, CAN' \
    or market == 'ALBERTA WEST, AB, CAN' or market == 'HUDSON VALLEY - POUGHKEEPSIE, NY' or market == 'FORT MCMURRAY, AB, CAN' \
    or market == 'NOVA SCOTIA NORTH, NS, CAN' or market == 'WATERTOWN, NY' or market == 'EDMUNSTON, NB, CAN' \
    or market == 'OLYMPIC NATIONAL PARK AREA, WA' or market == 'CEDAR CITY - BRYCE CANYON NATIONAL PARK, UT' or market == 'SUMMERSIDE, PEI, CAN' \
    or market == 'SALEM, OR (AREA)' or market == 'BUTTE - HELENA, MT' or market == 'BECKLEY, WV': return 'G5'
    
    elif market == 'SAGINAW, MI' or market == 'WHITE MOUNTAINS, NH' or market == 'FINGER LAKES NEW YORK' or market == 'NEW YORK - WEST' or market =='NEW BRUNSWICK NORTH, NB, CAN'  \
    or market == 'SASKATCHEWAN NORTH, SK, CAN' or market == 'MANITOBA SOUTH, MB, CAN' or market == 'BAR HARBOR, ME' \
    or market == 'WASHINGTON NORTHWEST' or market == 'VERMONT CENTRAL' or market == 'ALBERTA SOUTH, AB, CAN' \
    or market == 'WENATCHEE-LEAVENWORTH,WA' or market == 'GANANOQUE, ON, CAN' or market == 'PLATTSBURGH, (NY)' \
    or market == 'STRAITS OF MACKINAC, MI' or market == 'SANDUSKY, OH' or market == 'MANCHESTER, NH' \
    or market == 'ITHACA, NY' or market == 'MINNESOTA NORTHWEST' or market == 'MISSOULA, MT (AREA)' \
    or market == 'YELLOWKNIFE, NT, CAN' or market == 'PEI OTHER, PEI, CAN' or market == 'BILLINGS, MT (AREA)' \
    or market == 'RAPID CITY - MOUNT RUSHMORE, SD' or market == 'ANDOVER, MA'or market =='YUMA, AZ' or market == 'PORT HURON, MI' \
    or market == 'LANCASTER, PA' or market == 'FORT LEE - PARAMUS, NJ' or market == 'LEXINGTON, KY' \
    or market == 'MANITOBA NORTH, MB, CAN' or market == 'ABERDEEN - OCEAN SHORES, WA' or market == 'CANADA TERRITORIES, YT, CAN' \
    or market == 'KNOXVILLE, TN' or market == 'BURLINGTON - MOUNT VERNON, WA' or market == 'FREDERICKSBURG, VA' \
    or market == 'ARIZONA SOUTHEAST' or market == 'WINDHAM COUNTY, VT' or market == 'KENAI PENINSULA, AK' \
    or market == 'COLUMBIA, SC' or market == 'CALIFORNIA DESERTS SOUTH'  : return 'G6'
    
    elif market == 'ROME-UTICA, NY' or market == 'SANDPOINT, ID (AREA)' or market == 'MOAB - GREEN RIVER, UT' or market == 'WHITEHORSE, YT, CAN' or market =='NEW YORK - NORTHEAST'  \
    or market == 'SARNIA, ON, CAN' or market == 'AUGUSTA, ME' or market == 'LAKE POWELL - GLEN CANYON NATIONAL PARK, AZ' \
    or market == 'NEW YORK - SOUTH' or market == 'BISMARK, ND' or market == 'ALABAMA NORTH' \
    or market == 'ASHLAND - MEDFORD, OR' or market == 'WASHINGTON NORTHEAST' or market == 'THE BERKSHIRES, MA' \
    or market == 'EUGENE, OR (AREA)' or market == 'GRAND RAPIDS, MI' or market == 'WASHINGTON SOUTHWEST' \
    or market == 'LAKE HAVASU CITY, AZ' or market == 'SARATOGA, NY' or market == 'WYTHEVILLE - VA' \
    or market == 'ARCATA - EUREKA, CA' or market == 'SOUTH CAROLINA EAST' or market == 'CHATTANOOGA, TN' \
    or market == 'TOLEDO, OH' or market == 'IDAHO FALLS, ID' or market == 'WEST VIRGINIA SOUTH' \
    or market == 'SPRINGFIELD, MA' or market == 'OLYMPIA, WA' or market == 'GETTYSBURG - PA' : return 'G7'
    
    else : 'Others'
        
#testing for function srchbother_region_Hotel_type
Test.assertEquals(srchbNA_region_market('SAGINAW, MI'), 'G6','incorrect result: not matching')
Test.assertEquals(srchbNA_region_market('GRAVENHURST - BRACEBRIDGE - HUNTSVILLE, ON, CAN'), 'G1','incorrect result: not matching')


#Registering functions to UDF's
udfsrchbNAdf_region_star_rating=udf(srchbNAdf_region_star_rating, StringType())
udfsrchbNA_region_Hotel_type=udf(srchbNA_region_Hotel_type, StringType())
udfsrchbNA_region_market=udf(srchbNA_region_market, StringType())

#adding these UDF's as columns for existing dataframes
NAdf = NAdf.withColumn("star_rating_bucket", udfsrchbNAdf_region_star_rating("star_rating"))
NAdf = NAdf.withColumn("hotel_type_bucket", udfsrchbNA_region_Hotel_type("Hotel_type"))
NAdf = NAdf.withColumn("market_bucket", udfsrchbNA_region_market("market"))
NAdf = NAdf.withColumn("city_bucket", lit("NA"))
NAdf = NAdf.withColumn("region_bucket", lit("NA"))
NAdf = NAdf.withColumn("model_name", lit("North America"))

# registering dataframe as temporary tabel.
NAdf.registerTempTable("new_ytd_feb14to23jun15_NorthAmerica")







In [ ]:
## All super region tables completed need to do union of all tables.

In [ ]:
#union of all region tabels
cleaning_df = sqlContext.sql("select *  from new_ytd_feb14to23jun15_canada union all select * from new_ytd_feb14to23jun15_US \
                                   union all select *  from new_ytd_feb14to23jun15_Mexico union all select *  from new_ytd_feb14to23jun15_Caribbean \
                                   union all select *  from new_ytd_feb14to23jun15_NorthAmerica union all select *  from new_ytd_feb14to23jun15_NorthAmerica")

# registering dataframe as temporary tabel.
cleaning_df.registerTempTable("new_ytd_feb14to23jun15_Cleandata")




In [ ]:
## cleaning data is done

In [ ]:
## Random Forest AD Creation

In [ ]:
#practise work

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("k", StringType(), True)])

# or df = sc.parallelize([]).toDF(schema)
datesdf = sqlContext.createDataFrame([], schema)
#datesdf.show()


In [ ]:
from pyspark.sql import Row
#def date_generator():
import datetime
start = datetime.datetime.strptime("01-03-2015", "%d-%m-%Y")
end = datetime.datetime.strptime("15-03-2015", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
x = [date.strftime("%d-%m-%Y") for date in date_generated] 
    

y = [Row(tmp) for tmp in x]



In [ ]:
df = sqlContext.createDataFrame(y,["date"])

In [ ]:
from pyspark.sql.functions import *
df.select(from_unixtime(unix_timestamp(df.date,'dd-mm-yyyy')).alias("dates")).show()

In [ ]:

#Applying window function Rank to give ranking for rows

In [ ]:
#Example/practices

In [ ]:
dataframes1 = sqlContext.createDataFrame([{"Name" : "A", "Age" : 28},{ "Name" : "A", "Age" : 28},{"Name" : "C", "Age" : 23},{"Name" : "D", "Age" : 30},{"E" : "Vw", "Age" : 21}])

In [ ]:
#dataframes1.show()

dataframes1 = dataframes1.select("Name","Age")
#dataframes1.show()


In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import count



#w = Window.partitionBy("Name").orderBy(desc("Age"))

#dataframes1.select("Name",rank("Age").over(w).alias("HI")).show()

#rank1 = rank.over(w)

#window = Window.partitionBy("name").orderBy("age")

w = Window.orderBy(desc("Age"))

from pyspark.sql.functions import rank, min

#ranking = 

dataframes1.select("Age",rank().over(w).alias("rank")).show()



In [ ]:
#metabml data

In [ ]:
path1 ="/home/affine/vishnu/meta_BML_250_rows.csv"
meta_bml_df = sqlCtx.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',delimiter=',').load(path1)
#meta_bml_df.show(5)

In [ ]:
#meta_bml_df.show()

In [ ]:
#import sys
#from pyspark.sql.window import Window
#from pyspark.sql.functions import sum as sum_

meta_bml_df_1 = meta_bml_df

meta_bml_df_1 = meta_bml_df_1.filter((meta_bml_df_1['meta_site'] == 'TRIPADVISOR')  \
                                     & (meta_bml_df_1['point_of_sale'] == "CA") & (meta_bml_df_1['seller'] == "EXPEDIA"))


#meta_bml_df_1 = meta_bml_df.select("meta_site","point_of_sale","expedia_hotel_id","seller_rank")

#meta_bml_df_1 = meta_bml_df_1.groupBy("meta_site","point_of_sale","expedia_hotel_id","seller_rank").count()

#meta_bml_df_1 = meta_bml_df_1.withColumnRenamed('count', 'frequency')

#meta_bml_df_1.show()

#w = (Window()
    #.partitionBy(meta_bml_df_1.expedia_hotel_id, meta_bml_df_1.seller_rank)
    #.rowsBetween(-sys.maxsize, sys.maxsize))
#meta_bml_df_1 = meta_bml_df_1.withColumn("frequency", count(meta_bml_df_1.seller_rank).over(w))

#meta_bml_df_1.select('meta_site',"expedia_hotel_id" == 'null').show(100)
meta_bml_df_1.show()

